# Training Potts Models with Contrastive Divergence for Protein Design

## GREMLIN

https://github.com/whbpt/GREMLIN_PYTORCH/blob/master/GREMLIN_pytorch.ipynb

Also see https://github.com/HussainAther/potts/blob/main/notebooks/CSC412_Potts_Proteins_hussain.ipynb

### bmDCA

**Important Notes:**

*  All amino acids must be upper case

https://github.com/ranganathanlab/bmDCA

In [ ]:
!git clone https://github.com/ranganathanlab/bmDCA.git

Cloning into 'bmDCA'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 1679 (delta 15), reused 22 (delta 10), pack-reused 1646
Receiving objects: 100% (1679/1679), 790.26 KiB | 12.35 MiB/s, done.
Resolving deltas: 100% (1152/1152), done.


In [ ]:
!sudo apt-get update
!sudo apt-get install git gcc g++ automake autoconf pkg-config \
  libarmadillo-dev libopenblas-dev libarpack++2-dev

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [ ]:
%cd bmDCA
!bash autogen.sh --prefix=/usr/local && \
%cd ..

/content/bmDCA/bmDCA
+ ./configure --prefix=/usr/local
checking build system type... x86_64-pc-linux-gnu
checking host system type... x86_64-pc-linux-gnu
checking for a BSD-compatible install... /usr/bin/install -c
checking whether build environment is sane... yes
checking for a thread-safe mkdir -p... /bin/mkdir -p
checking for gawk... no
checking for mawk... mawk
checking whether make sets $(MAKE)... yes
checking whether make supports nested variables... yes
checking whether to enable maintainer-specific portions of Makefiles... yes
checking whether make supports nested variables... (cached) yes
checking for g++... g++
checking whether the C++ compiler works... yes
checking for C++ compiler default output file name... a.out
checking for suffix of executables... 
checking whether we are cross compiling... no
checking for suffix of object files... o
checking whether we are using the GNU C++ compiler... yes
checking whether g++ accepts -g... yes
checking for style of include used by mak

In [ ]:
%%shell
cd bmDCA
make -j4 && \
make install
cd ..

/bin/bash: line 0: cd: bmDCA: No such file or directory
Making all in src
make[1]: Entering directory '/content/bmDCA/bmDCA/src'
  CXX      bmdca.o
  CXX      model.o
  CXX      msa_stats.o
  CXX      msa.o
  CXX      run.o
  CXX      mcmc.o
  CXX      mcmc_stats.o
  CXX      graph.o
  CXX      utils.o
run.cpp: In constructor ‘Sim::Sim(MSAStats, std::__cxx11::string, std::__cxx11::string, bool)’:
run.cpp:383:10: warning: ignoring return value of ‘int chdir(const char*)’, declared with attribute warn_unused_result [-Wunused-result]
     chdir(dest_dir.c_str());
     ~~~~~^~~~~~~~~~~~~~~~~~
  CXX      bmdca_sample.o
  CXX      generator.o
  CXX      arma_convert.o
bmdca_sample.cpp: In function ‘int main(int, char**)’:
bmdca_sample.cpp:124:10: warning: ignoring return value of ‘int chdir(const char*)’, declared with attribute warn_unused_result [-Wunused-result]
     chdir(dest_dir.c_str());
     ~~~~~^~~~~~~~~~~~~~~~~~
  CXX      fasta_convert.o
  CXXLD    bmdca
  CXXLD    bmdca_sample
 

In [ ]:
!mkdir results

In [ ]:
%cd /content

/content


#### Training

100-245 of LCC?

In [ ]:
import numpy as np

def read_fasta(fname):
    seqs = []
    s = ""
    with open(fname) as f:
        line = f.readline()
        while line:
            if line.startswith(">"):
                if s != "":
                    seqs.append(list(s))
                s = ""
            elif len(line) > 0:
                s += line.strip()
            line = f.readline()
        seqs.append(list(s))
    return np.array(seqs)

In [ ]:
# seqs = read_fasta("pfam_hits.txt")
seqs = read_fasta("lcc_pfam.txt")

In [ ]:
mask = np.zeros(len(seqs[0]), dtype=np.bool)
for i in range(len(seqs[0])):
    gaps = 0
    for s in seqs:
        if s[i] == '-':
            gaps += 1
    if gaps/len(seqs) < 0.67:   # keep positions where less than 2/3rd are gaps
        mask[i] = True
seqs = seqs[:,mask]

In [ ]:
towrite = ""
for i in range(len(seqs)):
    if sum(seqs[i][100:] == "-") < 50:  # don't take sequences with many gaps
        towrite += ">{}\n".format(i)
        seq = "".join(seqs[i][:]) + "\n"
        towrite += seq.upper()   
with open("lcc_short.fasta",'w') as f:
    f.write(towrite)

In [ ]:
%%shell
rm results/*
bmdca -i lcc_short.fasta -r -d /content/results

15047 sequences
227 positions
21 amino acids (including gaps)
9318.1 effective sequences
initializing run... 5.29271 sec

Step: 1
sampling model with mcmc... 7.80018 sec
updating mcmc with samples... 4.8794 sec
computing sequence energies and correlations... 0.3637 sec
computing mcmc 1p and 2p statistics... 2.44098 sec
computing error and updating gradient... 1.53069 sec
update learning rate... 0.227634 sec
updating parameters... 0.230206 sec

Step: 2
sampling model with mcmc... 7.64549 sec
updating mcmc with samples... 4.84465 sec
computing sequence energies and correlations... 0.370797 sec
computing mcmc 1p and 2p statistics... 2.48229 sec
computing error and updating gradient... 1.54458 sec
update learning rate... 0.223336 sec
updating parameters... 0.220518 sec

Step: 3
sampling model with mcmc... 7.638 sec
updating mcmc with samples... 4.85864 sec
computing sequence energies and correlations... 0.367031 sec
computing mcmc 1p and 2p statistics... 2.4813 sec
computing error and upda

In [ ]:
!tar -czf boltzmann.tar.gz results/*

#### Sampling

Change temperature in a config file

In [ ]:
%%shell
bmdca_sample -p parameters.txt -d /content/results -o samples.txt -c config.conf

In [ ]:
!perl convert.pl lcc_pfam.txt lcc_pfam.fa


Error: txt is not a valid input format option


#### Likelihoods

Doesn't come with a function, so implement it ourselves by using the learned parameters $\mathbf{h}, \mathbf{J}$.

In [23]:
AA = ['a', 'r', 'n', 'd', 'c', 'q', 'e', 'g', 'h', 'i', 
      'l', 'k', 'm', 'f', 'p', 's', 't', 'w', 'y', 'v', '-']
AA_IDX = {AA[i]:i for i in range(len(AA))}
IDX_AA = {i:AA[i].upper() for i in range(len(AA))}

L = 6   # sequence length
q = 21  # number of tokens  (this is 'A' in the GREMLIN code)

h = np.random.randn(L*q)           # fields    (this is b0 in GREMLIN)
J = np.random.randn(L*q, L*q)      # couplings (this is W0 in GREMLIN)


# compute the energy logexp(.) for a sequence s

def logE(s, h, J):
    """
    s: sequence
    h: fields vector (L*q)
    J: couplings matrix (L*q, L*q)
    """
    sum_h = 0
    sum_J = 0
    for i, aa in enumerate(s):
        sum_h += h[i*q + aa]
    for i, aa1 in enumerate(s):
        for j, aa2 in enumerate(s):
            sum_J += J[i*q + aa1, j*q + aa2]
    return sum_h + sum_J

# convert list of M sequences of length L each into an MxL matrix of numbers in [0,20]

def seq2num(s):
    nums = []
    for aa in s:
        nums.append(AA_IDX[aa.lower()])
    return nums

def get_numerical_aas(fname):
    mat = []
    s = ""
    with open(fname) as f:
        line = f.readline()
        while line:
            if line.startswith(">"):
                if s != "":
                    mat.append(seq2num(s))
                s = ""
            elif len(line) > 0:
                s += line.strip()
            line = f.readline()
        mat.append(seq2num(s))
    return np.array(mat)

get_numerical_aas("test.fasta")

array([[12,  2, 13, 14,  1,  0],
       [15,  5, 10, 11,  6,  2]])

Get the in-distribution sequences (this is just the training data)

In [22]:
import numpy as np

in_dist = get_numerical_aas("test.fasta")

Get the out-of-distribution sequences. We construct these sequences in the following ways:



*   Let $S$ be an in-distribution sequence. We take $S'$ to be the shuffled $S$ which is likely out-of-distribution.
*   Let $S=S_1S_2$ be an in-distribution sequence. We assume that $S''=S_2S_1$ is out-of-distribution. This corresponds to swapping the two ends of the protein chain which is likely to disrupt the protein fold.



In [24]:
out_dist1 = np.array([np.random.permutation(s) for s in in_dist])

In [25]:
out_dist2 = np.array([np.concatenate((s[int(L/2):], s[:int(L/2)])) for s in in_dist])

Compute $\log \frac{P(S)}{P(S')}$

In [27]:
for i in range(len(in_dist)):
    print(logE(in_dist[i], h, J) - logE(out_dist1[i], h, J))

-13.234955596679765
12.768678655272508


Compute $\log \frac{P(S)}{P(S'')}$

In [28]:
for i in range(len(in_dist)):
    print(logE(in_dist[i], h, J) - logE(out_dist2[i], h, J))

-5.546759734304293
-11.50932788954155


### Contrastive Divergence

### Model evaluation

In [ ]:
!git clone https://github.com/igemto-drylab/CSBERG-ML.git
%cd CSBERG-ML
from util import *